In [14]:
import os 
import json
import itertools
import numpy as np
import csv
import cv2

In [32]:
cnts = 0
for idx,j in enumerate(os.listdir('mris_seg/images/ann')):
    with open('mris_seg/images/ann/' + j) as json_file:
        content = json.load(json_file)
        if (cnts < 1):
            if (len(content['objects']) > 0):
                cnts += 1
                print(content)

{'description': '', 'tags': [], 'size': {'height': 512, 'width': 512}, 'objects': [{'description': '', 'bitmap': None, 'tags': [], 'classTitle': 'vertebral_bone', 'points': {'exterior': [[241, 13], [246, 31], [237, 55], [277, 55], [298, 58], [304, 17], [280, 15]], 'interior': []}}, {'description': '', 'bitmap': None, 'tags': [], 'classTitle': 'vertebral_bone', 'points': {'exterior': [[300, 66], [283, 74], [267, 64], [246, 67], [235, 62], [232, 80], [220, 102], [248, 110], [272, 113], [286, 118], [293, 118], [295, 97]], 'interior': []}}, {'description': '', 'bitmap': None, 'tags': [], 'classTitle': 'vertebral_bone', 'points': {'exterior': [[290, 128], [282, 146], [278, 168], [278, 179], [251, 170], [227, 167], [209, 159], [207, 153], [217, 142], [219, 125], [219, 120], [231, 122], [261, 123]], 'interior': []}}, {'description': '', 'bitmap': None, 'tags': [], 'classTitle': 'vertebral_bone', 'points': {'exterior': [[193, 173], [194, 191], [182, 214], [199, 223], [225, 227], [247, 232], [2

In [38]:
def get_dicts():
    full_list = []
    for idx, j in enumerate(os.listdir('mris_seg/images/ann')):
        with open('mris_seg/images/ann/' + j) as json_file:
            content = json.load(json_file)
            if (len(content['objects']) > 0):
                # we have something...
                record = {}
                record["file_name"] = 'mris_seg/images/ann/' + j 
                record['height'] = content['size']['height']
                record['width'] = content['size']['width']
                objs = [] 
                for obj in content['objects']:
                    px = [p[0] for p in obj['points']['exterior']]
                    py = [p[1] for p in obj['points']['exterior']]
                    poly = [(x, y) for x, y in zip(px, py)]
                    poly = list(itertools.chain.from_iterable(poly))
                    o = {
                        "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                        "category_id": 0,
                        "iscrowd": 0,
                        "segmentation": [poly]
                    }
                    objs.append(o)
                record["annotations"] = objs
                full_list.append(record)
    return full_list # list[dict]

In [40]:
len(get_dicts())

40

In [36]:
def get_dicts_csv():
    full_list = []
    for idx, j in enumerate(os.listdir('masks_marks/')):
        with open('masks_marks/' + j) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=",")
            
            # we have something...
            img_filename =  '_'.join(j.split('_')[:-1])
            img = cv2.imread('images/{}.png'.format(img_filename))
            if (img is not None):
                record = {}
                record["file_name"] = 'images/' + img_filename + '.png'
                record['height'] = img.shape[0]
                record['width'] = img.shape[1]
                objs = [] 
                pts = []
                for idx_2,r in enumerate(csv_reader):
                    if (idx_2 > 0):
                        p_x, p_y = ([int(float(x)) for x in r[5:7]])
                        pts.append((p_x, p_y))
                    if (idx_2 % 6 == 0):
                        if (len(pts) == 6):
                            px = [p[0] for p in pts]
                            py = [p[1] for p in pts]
                            poly = [(x, y) for x, y in zip(px, py)]
                            poly = list(itertools.chain.from_iterable(poly))
                            o = {
                                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                                "category_id": 0,
                                "iscrowd": 0,
                                "segmentation": [poly]
                            }
                            objs.append(o)
                            pts = []
                record["annotations"] = objs
                full_list.append(record)
    return full_list # list[dict]

In [37]:
get_dicts_csv()

[{'file_name': 'images/MR_1056_L.png',
  'height': 512,
  'width': 512,
  'annotations': [{'bbox': [104, 211, 135, 250],
    'category_id': 0,
    'iscrowd': 0,
    'segmentation': [[117,
      250,
      125,
      240,
      135,
      231,
      121,
      211,
      114,
      218,
      104,
      223]]},
   {'bbox': [81, 183, 115, 218],
    'category_id': 0,
    'iscrowd': 0,
    'segmentation': [[91,
      218,
      103,
      211,
      115,
      207,
      108,
      183,
      97,
      190,
      81,
      197]]},
   {'bbox': [70, 152, 106, 186],
    'category_id': 0,
    'iscrowd': 0,
    'segmentation': [[75,
      186,
      89,
      181,
      106,
      178,
      102,
      152,
      87,
      156,
      70,
      161]]},
   {'bbox': [73, 116, 103, 146],
    'category_id': 0,
    'iscrowd': 0,
    'segmentation': [[73,
      146,
      86,
      145,
      102,
      145,
      103,
      116,
      88,
      119,
      73,
      120]]},
   {'bbox': [73, 82, 106, 1